In [11]:
import pandas as pd
import re
import csv
import os

In [12]:
data_path = "data-voltages"

In [13]:
def extract_data(file_path):
  """
  Extracts relevant data from a .dat file.

  Args:
    file_path: The path to the .dat file.

  Returns:
    A dictionary containing the extracted data.
  """

  data = {}
  with open(file_path, 'r') as f:
    for line in f:
      # Extract System Number (assuming it's after 'SerialNumber = ')
      match = re.search(r'System Serial Number = (\d+)', line)
      if match:
        data['System Number'] = match.group(1)

      # Extract Slot and Zone (assuming they're in lines like '> Zone: 12  Slot: 12')
      match = re.search(r'> Zone:\s*(\d+)\s+Slot:\s*(\d+)', line)
      if match:
        data['Zone'] = match.group(1)
        data['Slot'] = match.group(2)

      # Extract Test Step (assuming it's in lines like '***** Test Step: Power_Up *****')
      match = re.search(r'***** Test Step:\s*([\w_]+)\s*****', line)
      if match:
        data['Test Step'] = match.group(1)

      # Extract S_VSENSE and S_OVP (you'll need to provide the exact pattern)
      # Example (replace with your actual pattern):
      match = re.search(r'S_VSENSE\s*=\s*([\d\.]+)\s+S_OVP\s*=\s*([\d\.]+)', line)
      if match:
        data['S_VSENSE'] = match.group(1)
        data['S_OVP'] = match.group(2)

    return data

In [14]:
def process_files(dat_files, csv_file):
  """
  Processes multiple .dat files and writes extracted data to a CSV file.

  Args:
    dat_files: A list of paths to .dat files.
    csv_file: The path to the output CSV file.
  """
  with open(csv_file, 'w', newline='') as csvfile:
    fieldnames = ['System Number', 'Zone', 'Slot', 'Test Step', 'S_VSENSE', 'S_OVP']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for dat_file in dat_files:
      data = extract_data(dat_file)
      if data:  # Only write if data was extracted
        writer.writerow(data)

In [15]:
dat_file_list = [os.path.join(data_path, "Breithorn", "BRH_C1_HTOL7_T504_S414_CTRL4067_act12_14.dat")]  # Replace with your file list
csv_output = "output.csv"

process_files(dat_file_list, csv_output)